In [1]:
import os, sys
!{sys.executable} -m pip install -r requirements.txt

You should consider upgrading via the 'C:\Users\Stefa\.conda\envs\python36\python.exe -m pip install --upgrade pip' command.


In [1]:
from neo4j import GraphDatabase
from dotenv import dotenv_values
from soup_models import Document
from collections import defaultdict

# config = dotenv_values(".env")  # config = {"USER": "foo", "EMAIL": "foo@example.org"}

C:\Users\Stefa\.conda\envs\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Stefa\.conda\envs\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\Stefa\.conda\envs\python36\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
from graph_driver import GraphDBDriver

In [3]:
# greeter = HelloWorldExample("bolt://localhost:7687", "neo4j", "neo4j")
print("Testing Graph Driver...")
driver = GraphDBDriver()
print("Querying all nodes")
ret = driver.query("MATCH ()-[edge]->() WHERE ID(edge)=2 RETURN edge")
res = driver.record_to_models(ret[0])
res
# print("Uploading one node")
# tweet1 = Document("tweet1", "title", "tweet")
# source = Source("source1", "title", "source")
# driver.upload_nodes([tweet1, source])
# print("Adding connecting Edge")
# edge = Interacts(source, tweet1, "follows")
# driver.upload_edges([edge])
# # driver.close()

# print("Querying single node")
# result = driver.query_node(tweet1)
# print(result[0]['node'].keys())
# result1 = driver.query_edge(edge)
# print(result1[0]['edge'].keys())
# # print("Uploading several nodes with pre-existing in database")
# # tweets = [tweet, Document("test2", "title", "tweet"), Document("test3", "title", "tweet")]
# # # driver.upload_nodes(tweets)
# driver.close()
# print("Finished")

Testing Graph Driver...
Querying all nodes


{'edge': ('interacts', 'source1', 'tweet1')}

In [15]:
driver = GraphDBDriver()
ret = driver.query("MATCH (n:document) RETURN n.text AS resp")

In [104]:
import pickle
# texts = [record['resp'] for record in ret]
# pickle.dump(texts, open("data/texts.pkl", "wb"))
id_ = 5
texts = pickle.load(open("{}{}.pkl".format('data/', id_), "rb" ))
file = {
    'id': id_,
    'title': "Physical & Mental Health",
    'texts': texts
}
print(file)
pickle.dump(file, open("{}{}.pkl".format('data/', id_), "wb" ))

{'id': 5, 'title': 'Physical & Mental Health', 'texts': ['Where climate, jobs and health meet: The outdoors https://t.co/TrZ1Em7UZu @ProtectWinters @patagonia @REI #outdoors #ClimateAction #healthandwellness', 'Spending time outdoors is good for your physical and mental health. Here are three ways to add nature-based mindfulness to your routine from our friends at @rei:\n\nhttps://t.co/O8i8tmyaU9 https://t.co/BiznCkFuGv', 'So I\'ve been vegan for over a year now. For health reasons. Didn\'t realize that you could buy "vegan" shoes. Was browsing @rei tonight and saw that as a filter #vegan #allthethings', "It's #MentalHealthMonday, and it can't be expressed enough what an impact activity can have on your mental health! That's why we're excited to host an All Abilities Fitness Class with @REI this Wednesday, May 26; register now to join our Zoom livestream: https://t.co/cI0bLvtL0l https://t.co/jYWHIrNwJ0", '@REI But dogs are allowed at Cabelas, Lowe’s, Home Depot, etc. with no health pro

In [29]:
from collections import Counter
import spacy
import re

def combineDocs(docList):
    return " \n|=|\n ".join(docList)

def countWords(string):
    words = re.findall(r'\w+', string)
    return Counter(words)

nlp = spacy.load('en_core_web_sm')


big_doc = combineDocs(texts)

doc = nlp(big_doc)
counter = countWords(big_doc)
# counter.most_common(50)

In [83]:
nouns = [token for token in doc if token.pos_ in ["NOUN", "PROPN"]]
noun_count = Counter([token.text for token in nouns])

In [84]:
noun_count.most_common(100)

[('|=|', 844),
 ('@REI', 405),
 ('#', 117),
 ('REI', 70),
 ('amp', 69),
 ('store', 49),
 ('_', 45),
 ('️', 43),
 ('CooperativeAction', 41),
 ('sale', 40),
 ('climate', 39),
 ('action', 39),
 ('Network', 39),
 ('transportation', 39),
 ('TrailsTransform', 39),
 ('@Rei', 34),
 ('gear', 28),
 ('’s', 28),
 ('Thanks', 27),
 ('day', 27),
 ('stores', 25),
 ('time', 24),
 ('people', 24),
 ('@rei', 24),
 ('hiking', 23),
 ('？', 19),
 ('today', 19),
 ('year', 19),
 ('質問箱', 18),
 ('way', 18),
 ('ゆいあず', 18),
 ('😈', 18),
 ('@WickedRetro', 18),
 ('employees', 18),
 ('@AnniqueFrancois', 17),
 ('@SonyAlpha', 16),
 ('@Sigma_Photo', 16),
 ('@ARTOLOGY_US', 16),
 ('@RebeccaLMahan', 16),
 ('@itsmuzza2004', 16),
 ('bike', 16),
 ('%', 15),
 ('website', 14),
 ('shoes', 14),
 ('！', 14),
 ('boots', 13),
 ('week', 13),
 ('masks', 13),
 ('app', 13),
 ('years', 13),
 ('customer', 13),
 ('print', 13),
 ('experience', 12),
 ('items', 12),
 ('members', 12),
 ('trail', 12),
 ('mask', 12),
 ('card', 12),
 ('-', 11),
 ('s

In [114]:
# doc.text
tweets = pickle.load(open("data/texts.pkl", "rb" ))

In [126]:
mockNarratives = {
    0: {
        'id': 0,
        'title': 'Shopping at the REI Store',
        'sentiment': 'good',
        'dominance': 120,
    },
    1: {
        'id': 1,
        'title': 'Gear for Outdoor Activity',
        'sentiment': 'good',
        'dominance': 135,
    },
    2: {
        'id': 2,
        'title': 'The #CooperativeAction Network',
        'sentiment': 'neutral',
        'dominance': 39,
    },
    3: {
        'id': 3,
        'title': 'COVID & Mask-wearing',
        'sentiment': 'divided',
        'dominance': 15,
    },
    4: {
        'id': 4,
        'title': 'Problems with Mobile App',
        'sentiment': 'bad',
        'dominance': 9,
    },
    5: {
        'id': 5,
        'title': 'Physical & Mental Health',
        'sentiment': 'good',
        'dominance': 13,
    },
}

narrative_templates = {
    0 : ["store", "sale", "website"], # REI stores are great
    1 : ["hiking", "hike", "gear", "outdoor", "biking"], # Gear for Outdoor Activity
    2 : ["climate", "cooperativeaction"], # The #CooperativeAction Network
    3 : ["masks", "covid"], # COVID & Mask-wearing
    4 : [" app ", " app."], # Problems with Mobile App
    5 : ["health", "mental"], # Physical & Mental Health
}

keywords = narrative_templates[5]
matches = []
for word in keywords:
    matches.extend([tweet for tweet in tweets if word in tweet])
matches
# pickle.dump(matches, open("data/5-texts.pkl", "wb"))

['Where climate, jobs and health meet: The outdoors https://t.co/TrZ1Em7UZu @ProtectWinters @patagonia @REI #outdoors #ClimateAction #healthandwellness',
 'Spending time outdoors is good for your physical and mental health. Here are three ways to add nature-based mindfulness to your routine from our friends at @rei:\n\nhttps://t.co/O8i8tmyaU9 https://t.co/BiznCkFuGv',
 'So I\'ve been vegan for over a year now. For health reasons. Didn\'t realize that you could buy "vegan" shoes. Was browsing @rei tonight and saw that as a filter #vegan #allthethings',
 "It's #MentalHealthMonday, and it can't be expressed enough what an impact activity can have on your mental health! That's why we're excited to host an All Abilities Fitness Class with @REI this Wednesday, May 26; register now to join our Zoom livestream: https://t.co/cI0bLvtL0l https://t.co/jYWHIrNwJ0",
 '@REI But dogs are allowed at Cabelas, Lowe’s, Home Depot, etc. with no health problems or code violations. Sounds like poor excuses t

In [117]:
tweets

['@REI Hoping @REI will run more promo more often! https://t.co/h11o0liqBk',
 '@REI I was looking for men hiking boots if there is anything on sale.',
 '@MaddyDono @MassDCR YES! Also makes the type hard to read for us olds. Please @MassDCR get a college kid to redo your maps with the $ @rei is paying for their logo on your maps.',
 '@REI Eventually the hate crews will only have each other to target and they’ll eliminate each other in court and street fights while the rest of us move BEYOND LEGALISM',
 '@REI Way easier to just go camp on private land that isn’t patrolled by colorist rapey fascists who like doing supposedly legal terrorism',
 '@REI I wouldn’t even go backpacking again after seeing how some of your fans act on social media. Trust',
 '@REI still waiting on a response f yesterday \n\nI work in internal development (so it was not exactly a request) &amp; I can become less than civil in making that request again \n\nFiling a foia to get info will give me WAY MORE problems tha

In [53]:
for node_type in ["document", "entity", "action", "source"]:
    querystring = "CREATE CONSTRAINT unique_key_{} IF NOT EXISTS ON (n:{}) ASSERT n.key IS UNIQUE".format(node_type, node_type)
    print(driver.query(querystring))

[]
[]
[]
[]


In [54]:
driver.query_node(tweet)

[<Record node=<Node id=16 labels=frozenset({'document'}) properties={'details': 'hello!', 'doc_type': 'tweet', 'type': 'document', 'date_processed': neo4j.time.DateTime(2021, 5, 22, 16, 42, 13.227833), 'title': 'title', 'key': 'test1'}>>]

In [55]:
driver.upload_nodes(tweets)

test1 already exists in database
test2 already exists in database
test3 already exists in database


[]